

<div style=" text-align: center">

<div style="margin-bottom: -20px">
    <img style="display: inline; width: 200px" src="https://www.saashub.com/images/app/service_logos/239/26gdror9vj3e/medium.png?1679110815" />
&nbsp;&nbsp;  <span style="font-weight: 100">x</span>  &nbsp;&nbsp;&nbsp;&nbsp;
    <img style="display: inline; width: 200px" src="https://raw.githubusercontent.com/PySport/kloppy/master/docs/logo.png" />
        </div>
<!--     <h1>Kloppy <span style="font-weight: 100">x</span> DuckDB</h1> -->
    <br />
        <h6 style="width: 800px; margin: auto">Unlocking Valuable Sports Insights: Analyzing SportsCode XML Files with DuckDB and Kloppy</h6>
    <br />
    <span style="font-size: 20px">Koen Vossen - PySport Meetup Eindhoven - April 2023</span>
    </div>






## Content

- Introduction
- DuckDB
- Kloppy
- Code
- Conclusion

# Introduction

- founder TeamTV
- korfbal coach
- founder PySport

# DuckDB

DuckDB is an in-process SQL OLAP database management system

<img src="duckdb.png" style="width: 50%; margin: auto" />

# In-process database vs External database

<img src="duckdb2.png?t=3" style="width: 90%; margin: auto" />

# In-process database: SQLite vs DuckDB

<img src="sqlite-duckdb.png" style="width: 90%; margin: auto" />

# Kloppy

Standardizing soccer tracking and event data

#### Loading data
- Load **public datasets** to get started right away
- Understandable **standardized data models** for tracking and event data
- Out-of-the-box **(de)serializing** tracking and event data from different vendors into standardized models and vice versa

#### Processing data
- Flexibly transform a dataset's **pitch dimensions** from one format to another (e.g., from OPTA's 100x100 to TRACAB meters)
- Transform the **orientation** of a dataset (e.g., from TRACAB fixed orientation to "Home Team" orientation)

<img src="data-logger.jpeg" style="width: 80%; margin: auto" />

<img src="what-kloppy-does.png" style="width: 100%; margin: auto" />

<img src="shapes.png" style="width: 100%; margin: auto" />

# Code



In [ ]:
%pip install kloppy

In [ ]:
# Import sportscode deserializer
from kloppy import sportscode

In [ ]:
# Load the XML file into a kloppy dataset
dataset = sportscode.load("files/DBO - OR 2 (1).xml")


In [ ]:
# Show all records within the dataset
for record in dataset:
    print(f"{record.code} - {record.period.id} - {record.timestamp}")

# Lets do some pre processing

In [ ]:
from kloppy.domain import Period
dataset = sportscode.load("files/DBO - OR 2 (1).xml")

period = None
for record in dataset.records:
    if record.code == 'Start':
        if period:
            # End current period
            period.end_timestamp = record.timestamp - 0.001
            
        period = Period(
            id=period.id + 1 if period else 1,
            start_timestamp=record.timestamp,
            end_timestamp=None
        )
        
    if not period:
        # Data before first 'Period marker'
        continue

    record.period = period
    record.timestamp -= period.start_timestamp

In [ ]:
for record in dataset:
    print(f"{record.code} - {record.period.id} - {record.timestamp}")

# Lets do some data cleaning

In [ ]:
PLAYERS = [
    "Gijs", "Jamie", "Sander", "Jelle", "Sheldon", "Phil"
]

players_dataset = dataset.filter(lambda item: item.code in PLAYERS)

In [ ]:
for record in players_dataset:
    print(f"{record.code} - {record.period.id} - {record.timestamp}")

In [ ]:
# Convert kloppy Dataset into Pandas Dataframe
players_data = players_dataset.to_df()
players_data

# Lets use DuckDB

In [ ]:
# This requires DuckDB 0.7.2 (preview release)
%pip install duckdb==0.7.2.dev1381

In [ ]:
import duckdb
con = duckdb.connect()
con

In [ ]:
con.query('SELECT timestamp, code, "Duel +", "Duel -" FROM players_data WHERE "Duel +" OR "Duel -" ORDER BY period_id, timestamp LIMIT 10')

In [ ]:
con.query("""
    SELECT 
        code, period_id, COUNT("Duel +"), COUNT("Duel -") 
    FROM players_data WHERE "Duel +" OR "Duel -" GROUP BY ALL ORDER BY code

""")

# Some more data cleaning

In [ ]:
PLAYERS = [
    "Gijs", "Jamie", "Sander", "Jelle", "Sheldon", "Phil"
]

players_dataset = dataset.filter(lambda item: item.code in PLAYERS)

def onehot_to_cat(labels, mapping):
    for label_name, category in mapping.items():
        if labels.get(label_name):
            return category
    
def duel_encoder(record):
    return onehot_to_cat(
        record.labels,
        {
            "Duel +": "won",
            "Duel -": "lost"
        }
    )
    
players_data_cat = players_dataset.to_df(
    "period_id",
    player=lambda record: record.code,
    duel=duel_encoder
)


In [ ]:
players_data_cat

In [ ]:
con.query("""
    SELECT 
        player, period_id, 
        COUNT(*) FILTER(duel = 'won') as duel_won
    FROM players_data_cat WHERE duel IS NOT NULL GROUP BY ALL ORDER BY player

""")

In [ ]:
con.query("""
    SELECT
        
        player, period_id,
        printf('%.1f%%', (COUNT(*) FILTER(duel='won'))::DECIMAL / COUNT(*) * 100) as won_percentage,
    FROM players_data_cat
    WHERE duel IN ('won', 'lost')
    GROUP BY ALL
""")

In [ ]:
con.query("""
WITH players AS (
    SELECT
        code, 
        period_id,
        printf('%.1f%%', (COUNT(*) FILTER(duel='won'))::DECIMAL / COUNT(*) * 100) as duel_percentage
    FROM players_data_cat
    GROUP BY ALL
    WHERE duel IN ('won', 'lost')
)
PIVOT 
    players
ON period_id
USING FIRST(duel_percentage)
GROUP BY code
""")

# Read from multiple matches

In [ ]:
import os

PLAYERS = [
    "Gijs", "Jamie", "Sander", "Jelle", "Sheldon", "Phil"
]

def onehot_to_cat(labels, mapping):
    for label_name, category in mapping.items():
        if labels.get(label_name):
            return category

        
def duel_encoder(record):
    return onehot_to_cat(
        record.labels,
        {
            "Duel +": "won",
            "Duel -": "lost"
        }
    )


In [ ]:

def load_file(filename):
    dataset = sportscode.load(filename)

    period = None
    for record in dataset.records:
        if record.code == 'Start':
            if period:
                # End current period
                period.end_timestamp = record.timestamp - 0.001

            period = Period(
                id=period.id + 1 if period else 1,
                start_timestamp=record.timestamp,
                end_timestamp=None
            )

        if not period:
            # Data before first 'Period marker'
            continue

        record.period = period
        record.timestamp -= period.start_timestamp
        
    players_dataset = dataset.filter(lambda item: item.code in PLAYERS)
    
    match = os.path.basename(filename).split(" 2")[0]
    
    return players_dataset.to_df(
        "period_id", 
        match=match,
        player=lambda record: record.code,
        duel=duel_encoder
    )

In [ ]:
import glob

files = glob.glob("files/* 2*.xml")
files

In [ ]:
import pandas as pd
df = pd.concat(
    [load_file(file) for file in files]
)
df

In [ ]:
con.query("""
SELECT 
    match, 
    period_id, 
    COUNT(*) FILTER (duel='won') as won,
    COUNT(*) FILTER (duel='lost') as lost
FROM df WHERE duel IN ('won', 'lost')
GROUP BY ALL
ORDER BY match, period_id
""")

# Using football event data

In [ ]:
from kloppy import statsbomb

dataset = statsbomb.load_open_data()

df = (
    dataset
    .filter("pass")
    .to_df(
        "team",
        "player",
        "result",
    )
)

In [ ]:
df

In [ ]:
con.query("""
SELECT player, result, COUNT(*) FROM df GROUP BY ALL ORDER BY ALL
""")

In [ ]:
con.query("PIVOT df ON result GROUP BY team")

In [ ]:
df_shot = (
    dataset
    .filter("shot")
    .to_df(
        "team",
        "player",
        "result",
    )
)
results = con.query("PIVOT df_shot ON result GROUP BY team, player")
results

# Export to excel 😎

In [ ]:
%pip install openpyxl

In [ ]:
results.df().to_excel("output.xlsx", index=False)

In [ ]:
import subprocess
subprocess.call(["open", "."])

# Conclusion

- kloppy can be used to read sportscode xml files
- data cleaning/transformation can be done in a few lines of code
- cleaned kloppy dataset can be transformed into pandas DataFrame
- DuckDB can query pandas DataFrame directly

# Questions?